In [2]:
import pandas as pd
import numpy as np
from pathlib import Path


pd.set_option('display.expand_frame_repr', False)  # keine Umbrüche mehr
pd.set_option('display.max_columns', None)         # alle Spalten anzeigen

In [ ]:
# load and combine csv
base_path = Path('xml_srl_work_prices/pkl')

pkl_files = list(base_path.rglob('*.pkl'))

dfs = []

for file in pkl_files:
    df  = pd.read_pickle(file)
    df.set_index('Datum', inplace=True)
    dfs.append(df)


df_all = pd.concat(dfs, axis=0, ignore_index=False)
df_all.index = df_all.index - pd.offsets.Day(1)


df_all.index = df_all.index.tz_localize('UTC').tz_convert('Europe/Berlin')

erster_tag = df_all.index.normalize().min()
df_new = df_all[df_all.index.normalize() > erster_tag]

letzter_tag = df_all.index.normalize().max()

df_new_new = df_new[df_new.index.normalize() < letzter_tag]

print(df_new_new)
df_new_new.to_pickle('work_prices 2022-07-01 - 2025-06-29.pkl')



In [ ]:
# join prices with setpoints

df_srl_prices = pd.read_pickle('work_prices 2022-07-01 - 2025-06-29.pkl')
df_srl_power_setpoint = pd.read_pickle('srl_setpoint_4s.pkl')

print(len(df_srl_prices))
print(len(df_srl_power_setpoint))

# join
df_srl_work = df_srl_prices.join(df_srl_power_setpoint, how='inner')

df_srl_work.to_pickle('srl_work_4s.pkl')

In [ ]:
df_srl_prices = pd.read_pickle('work_prices 2022-07-01 - 2025-06-29.pkl')
df_srl_prices


,position,Price [EUR/MWh] Up,Price [EUR/MWh] Down
Datum,,,
2022-06-28 00:00:00+02:00,19801,NaN,147.870
2022-06-28 00:00:04+02:00,19802,228.694,228.694
2022-06-28 00:00:08+02:00,19803,350.860,NaN
2022-06-28 00:00:12+02:00,19804,350.860,NaN
2022-06-28 00:00:16+02:00,19805,364.540,NaN
...,...,...,...
2025-06-27 23:59:40+02:00,19796,NaN,77.270
2025-06-27 23:59:44+02:00,19797,NaN,77.000
2025-06-27 23:59:48+02:00,19798,NaN,75.500


In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from config import START_DATE, END_DATE, PATH_SRL_WORK_DATA


SPECIFIC_AGING_COSTS = 5  # €/MWh
SYSTEM_POWER = 1          # MW
MIN_SETPOINT =  0

df = pd.read_pickle(PATH_SRL_WORK_DATA)
df['work_target_neg'] = 0.0 
df['work_target_pos'] = 0.0

print((df))


mask_neg = (
    (-df['Price [EUR/MWh] Down'] > SPECIFIC_AGING_COSTS)
    & (df['Price [EUR/MWh] Up'] == 0)
    & (df['affr_neg_setpoint'] > MIN_SETPOINT)
)
df.loc[mask_neg, 'work_target_neg'] = np.minimum(SYSTEM_POWER, df['affr_neg_setpoint'].abs()) * 4 / 3600

mask_pos = ( 
    (df['Price [EUR/MWh] Up'] >  SPECIFIC_AGING_COSTS
     & (df['Price [EUR/MWh] Down'] == 0 )
     & (df['affr_pos_setpoint'] > MIN_SETPOINT))
)
df.loc[mask_pos, 'work_target_pos'] = np.minimum(SYSTEM_POWER, df['affr_pos_setpoint'].abs()) * 4 / 3600



In [ ]:
dfx = df.copy()

# new col profit_neg
dfx['profit_neg'] = dfx['prices_neg'] * dfx['work_target_neg']
# new col profit_pos
dfx['profit_pos'] = dfx['prices_pos'] * dfx['work_target_pos']


# durchschnitt 15min
dfx_15min = dfx.resample('15T').mean()

dfx_15min['price_neg_15min'] = dfx_15min['profit_neg'] / dfx_15min['work_target_neg']
dfx_15min['price_pos_15min'] = dfx_15min['profit_pos'] / dfx_15min['work_target_pos']


print(dfx_15min)

